In [18]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

In [ ]:
# amount of each categories for company bundle
Company_bundle = np.array([
    [5,3,1], # company A
    [7,3,2], # comapny B
    [2,1,1], # Company c
    [3,1,4] # company D
])

Company, bundle_profit, per_lb = gp.multidict({
    'A' : [2, 9],
    'B' : [3, 10],
    'C' : [1, 4],
    'D' : [2, 9]
})

# amount of supply, already subtracted from the company relationship require
# money get from small recycler 
Categories, amount, profit_single = gp.multidict({
    'paper' : [15000, 0.05],
    'glass' : [7000, 0.03],
    'plastic' : [14000, 0.02]
})


1) 

Total collection

15,000 pounds of paper

7,000 pounds of glass

14,000 pounds of plastic

In [137]:
# 
company_wegiht = np.array(list(per_lb.values()))
company_profit = np.array(list(bundle_profit.values()))
single_profit = np.array(list(profit_single.values()))
available_materials = np.array(list(amount.values()))

model_revenue = gp.Model(name='revenue_maximization')
model_revenue.setParam('OutputFlag', 1)

# Decision variables:
# 1. Number of bundles sold to each company (can be fractional after minimum 100)
bundles_sold = model_revenue.addMVar(len(Company), vtype=GRB.CONTINUOUS, lb=100, name='bundles_sold')

# 2. Materials sold to small recyclers
small_recycler_materials = model_revenue.addMVar(len(Categories), vtype=GRB.CONTINUOUS, lb=0, name='small_recycler')


# Material balance: for each material type (paper, glass, plastic)
# Total used = materials in bundles + materials to small recyclers
for j in range(len(Categories)):
    # Materials used in bundles to large companies
    materials_in_bundles = gp.quicksum(Company_bundle[i, j] * bundles_sold[i] for i in range(len(Company)))
    
    # Total materials used must equal available
    model_revenue.addConstr(
        materials_in_bundles + small_recycler_materials[j] == available_materials[j],
        name=f"material_balance_{list(Categories)[j]}"
    )

# OBJECTIVE: Maximize revenue

# Revenue from large recyclers (per bundle price * number of bundles)
large_recycler_revenue = company_profit @ bundles_sold

# Revenue from small recyclers (price per pound * pounds sold)
small_recycler_revenue = single_profit @ small_recycler_materials

# Total revenue
total_revenue = large_recycler_revenue + small_recycler_revenue

model_revenue.setObjective(total_revenue, GRB.MAXIMIZE)

# Solve
model_revenue.optimize()

# Display results
if model_revenue.status == GRB.OPTIMAL:
    print("\n" + "="*70)
    print("OPTIMAL SOLUTION FOUND!")
    print("="*70)
    
    bundles_vals = bundles_sold.X
    small_vals = small_recycler_materials.X
    
    print("\nBundles sold to large recyclers:")
    total_large_revenue = 0
    for i, comp in enumerate(Company):
        revenue = company_profit[i] * bundles_vals[i]
        total_large_revenue += revenue
        print(f"  Company {comp}: {bundles_vals[i]:.2f} bundles → ${revenue:.2f}")
    
    print(f"\n  Total revenue from large recyclers: ${total_large_revenue:.2f}")
    
    print("\nMaterials sold to small recyclers:")
    total_small_revenue = 0
    for j, cat in enumerate(Categories):
        revenue = single_profit[j] * small_vals[j]
        total_small_revenue += revenue
        print(f"  {cat.capitalize()}: {small_vals[j]:.2f} lbs → ${revenue:.2f}")
    
    print(f"\n  Total revenue from small recyclers: ${total_small_revenue:.2f}")
    
    print("\n" + "="*70)
    print(f"TOTAL REVENUE: ${model_revenue.objVal:.2f}")
    print("="*70)
    
    


Set parameter OutputFlag to value 1
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[x86] - Darwin 21.6.0 21H1320)

CPU model: Intel(R) Core(TM) i5-5350U CPU @ 1.80GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Optimize a model with 3 rows, 7 columns and 15 nonzeros
Model fingerprint: 0x40b7f183
Coefficient statistics:
  Matrix range     [1e+00, 7e+00]
  Objective range  [2e-02, 3e+00]
  Bounds range     [1e+02, 1e+02]
  RHS range        [6e+03, 1e+04]
Presolve time: 0.02s
Presolved: 3 rows, 7 columns, 15 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.7360000e+31   1.112500e+31   2.736000e+01      0s
       4    7.9476000e+03   0.000000e+00   0.000000e+00      0s

Solved in 4 iterations and 0.04 seconds (0.00 work units)
Optimal objective  7.947600000e+03

OPTIMAL SOLUTION FOUND!

Bundles sold to large recyclers:
  Company A: 100.00 bundles → $200.00
  Company B: 100.00 bundles → $300.00
  Company C: 1940